<a href="https://colab.research.google.com/github/mehrnaz1368/classification/blob/main/recommendation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt

In [3]:
movies_df= pd.read_csv('movies.csv')
ratings_df= pd.read_csv('ratings.csv')

In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# we want to move (year) from title column to independent column

In [5]:
movies_df['year']= movies_df.title.str.extract('(\(\d\d\d\d\))', expand=False)

#Remove the parentheses

In [6]:
movies_df['year']= movies_df.year.str.extract('(\d\d\d\d)', expand=False)

#remove the year from tiltle column

In [7]:
movies_df['title']= movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title']= movies_df['title'].apply(lambda x: x.strip())

<ipython-input-7-6735a105e9a6>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title']= movies_df.title.str.replace('(\(\d\d\d\d\))', '')


In [8]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


#drop the genres

In [9]:
movies_df = movies_df.drop('genres', 1)

<ipython-input-9-cec6dd96bd4d>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  movies_df = movies_df.drop('genres', 1)


In [10]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


#remove timestamp from ratings_df

In [11]:
ratings_df=ratings_df.drop('timestamp', 1)

<ipython-input-11-9b66ec7e4abf>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ratings_df=ratings_df.drop('timestamp', 1)


# Euclidean distance measurement
## relate every title to rate

---



In [13]:
userInput= [
    {'title': 'Breakfast Club, The', 'rating':5},
    {'title': 'Toy Story', 'rating':3.5},
    {'title': 'Jumanji', 'rating':2},
    {'title': "Puple Fiction", 'rating':5},
    {'title': 'Akira', 'rating':4.5},
    ]
inputMovies= pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Puple Fiction,5.0
4,Akira,4.5


#filter the movie by title

In [14]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies=pd.merge(inputId, inputMovies)
inputMovies= inputMovies.drop('year', 1)
inputMovies

<ipython-input-14-1374d88dfdf2>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  inputMovies= inputMovies.drop('year', 1)


,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,1274,Akira,4.5
3,1968,"Breakfast Club, The",5.0


#We want to isolate users who watched a movie with the person we followed

In [15]:
userSubset= ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
0,1,1,4.0
422,4,1968,4.0
516,5,1,4.0
560,6,2,4.0
874,7,1,4.5


# we want to group up the rows by user ID

In [18]:
userSubsetGroup= userSubset.groupby(['userId'])
userSubsetGroup.get_group(7)


,userId,movieId,rating
874,7,1,4.5


#sort users

In [19]:
userSubsetGroup= sorted(userSubsetGroup, key=lambda x: len(x[1]), reverse=True)
userSubsetGroup[0:3]

<ipython-input-19-fd77ebe283f0>:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup= sorted(userSubsetGroup, key=lambda x: len(x[1]), reverse=True)


[(91,
         userId  movieId  rating
  14121      91        1     4.0
  14122      91        2     3.0
  14316      91     1274     5.0
  14383      91     1968     3.0),
 (177,
         userId  movieId  rating
  24900     177        1     5.0
  24901     177        2     3.5
  25069     177     1274     2.0
  25129     177     1968     3.5),
 (219,
         userId  movieId  rating
  31524     219        1     3.5
  31525     219        2     2.5
  31628     219     1274     2.5
  31680     219     1968     3.0)]

# Pearson correlation with 100 user

In [54]:
userSubsetGroup= userSubsetGroup[0:100]
pearsonCorrelationDict = {}
for name, group in userSubsetGroup:
  group= group.sort_values(['movieId'], ascending=False)
  nRatings= len(group)
  temp_df = inputMovies[inputMovies['movieId']. isin(group['movieId'].tolist())]
  tempRatingList= temp_df['rating'].tolist()
  tempGroupList= group['rating']. tolist()
  Sxx= sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList), 2)/ float(nRatings)
  Syy= sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList), 2)/ float(nRatings)
  Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
else:
        pearsonCorrelationDict[name] = 0


In [56]:
pearsonCorrelationDict

{462: -1.6666666666666667}

In [55]:
pearsonCorrelationDict.items()

pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
print(pearsonDF.head())

   similarityIndex  userId
0        -1.666667     462


# sort pearson

In [57]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
print(topUsers.head())

   similarityIndex  userId
0        -1.666667     462


#let's start recommending movies top usres

In [58]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
print(topUsersRating.head())

   similarityIndex  userId  movieId  rating
0        -1.666667     462        1     1.5
1        -1.666667     462       10     3.0
2        -1.666667     462       16     3.5
3        -1.666667     462       21     4.0
4        -1.666667     462       25     3.0


# evaluate weighting top users

In [59]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
print(topUsersRating.head())

   similarityIndex  userId  movieId  rating  weightedRating
0        -1.666667     462        1     1.5       -2.500000
1        -1.666667     462       10     3.0       -5.000000
2        -1.666667     462       16     3.5       -5.833333
3        -1.666667     462       21     4.0       -6.666667
4        -1.666667     462       25     3.0       -5.000000


# applies a sum to the top Users after grouping it up by userId

In [60]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
print(tempTopUsersRating.head())


         sum_similarityIndex  sum_weightedRating
movieId                                         
1                  -1.666667           -2.500000
10                 -1.666667           -5.000000
16                 -1.666667           -5.833333
21                 -1.666667           -6.666667
25                 -1.666667           -5.000000


# Creates an empty dataframe

In [61]:
recommendation_df = pd.DataFrame()

# weighted average

In [62]:
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
print(recommendation_df.head())

         weighted average recommendation score  movieId
movieId                                                
1                                          1.5        1
10                                         3.0       10
16                                         3.5       16
21                                         4.0       21
25                                         3.0       25


# sort it and see the top 20 movies that recommended

In [66]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
print(recommendation_df.head(10))


         weighted average recommendation score  movieId
movieId                                                
168250                                     5.0   168250
858                                        5.0      858
6945                                       5.0     6945
1676                                       5.0     1676
152711                                     5.0   152711
1228                                       5.0     1228
1952                                       5.0     1952
1954                                       5.0     1954
8042                                       5.0     8042
6611                                       5.0     6611


,movieId,title,year
659,858,"Godfather, The",1972
929,1228,Raging Bull,1980
1261,1676,Starship Troopers,1997
1429,1952,Midnight Cowboy,1969
1431,1954,Rocky,1976
4474,6611,Umberto D.,1952
4646,6945,My Architect: A Son's Journey,2003
5094,8042,Mean Streets,1973
9234,152711,Who Killed Chea Vichea?,2010
9462,168250,Get Out,2017


In [67]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
659,858,"Godfather, The",1972
929,1228,Raging Bull,1980
1261,1676,Starship Troopers,1997
1429,1952,Midnight Cowboy,1969
1431,1954,Rocky,1976
4474,6611,Umberto D.,1952
4646,6945,My Architect: A Son's Journey,2003
5094,8042,Mean Streets,1973
9234,152711,Who Killed Chea Vichea?,2010
9462,168250,Get Out,2017
